# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Bredasdorp,-34.5322,20.0403,62.94,77,41,5.99,1021,ZA,1732911672
1,1,Dezhou,37.4486,116.2925,42.17,49,0,6.17,1017,CN,1732911673
2,2,Adamstown,-25.0660,-130.1015,72.84,80,100,15.95,1020,PN,1732911675
3,3,Tazovsky,67.4667,78.7000,9.84,93,100,10.56,1024,RU,1732911677
4,4,Puerto Natales,-51.7236,-72.4875,59.45,44,40,27.63,1009,CL,1732911678


In [12]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     553 non-null    int64  
 1   City        553 non-null    object 
 2   Lat         553 non-null    float64
 3   Lng         553 non-null    float64
 4   Max Temp    553 non-null    float64
 5   Humidity    553 non-null    int64  
 6   Cloudiness  553 non-null    int64  
 7   Wind Speed  553 non-null    float64
 8   Pressure    553 non-null    int64  
 9   Country     547 non-null    object 
 10  Date        553 non-null    int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 47.7+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
df2 = city_data_df.dropna()

# Create a mask using the cleaned DataFrame
mask = (df2["Max Temp"] >= min_temp) & (df2["Max Temp"] < max_temp) & (df2["Wind Speed"] < max_wind)

# Apply the mask to the cleaned DataFrame
df2 = df2.loc[mask].reset_index(drop=True)

# Display sample data
df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,5,Puerto Ayora,-0.7393,-90.3518,72.55,87,37,5.01,1010,EC,1732911680
1,27,Wailua Homesteads,22.0669,-159.3780,76.12,83,75,8.05,1017,US,1732911706
2,37,Muisne,0.6000,-80.0333,75.92,77,94,9.78,1009,EC,1732911718
3,39,Meulaboh,4.1363,96.1285,76.12,89,100,3.58,1007,ID,1732911721
4,40,Bontang,0.1333,117.5000,77.20,91,89,6.31,1006,ID,1732911722
...,...,...,...,...,...,...,...,...,...,...,...
82,515,Sisa,-6.6167,-76.6915,76.64,98,92,1.39,1009,PE,1732912500
83,523,Nkoteng,4.5167,12.0333,73.51,50,64,0.72,1013,CM,1732912510
84,532,Palopo,-2.9996,120.1921,75.65,92,73,5.61,1008,ID,1732912522
85,536,Prey Veng,11.4868,105.3253,71.60,68,76,8.75,1010,KH,1732912527


### Step 3: Create a new DataFrame called `hotel_df`.

In [63]:
city_data_df = pd.DataFrame({
    'city': ['Meulaboh', 'Muisne', 'Bontang'],
    'country': ['Indonesia', 'Equador', 'Indonesia'],
    'lat': [4.1363, 0.6000, 0.1333],
    'lng': [96.1285, -80.0333, 117.5000],
    'humidity': [30, 45, 50]
})

In [64]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['city', 'country', 'lat', 'lng', 'humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
print(hotel_df)

       city    country     lat       lng  humidity Hotel Name
0  Meulaboh  Indonesia  4.1363   96.1285        30       None
1    Muisne    Equador  0.6000  -80.0333        45       None
2   Bontang  Indonesia  0.1333  117.5000        50       None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000  # 10km
params =  params = {
        "categories": categories,
        "limit": limit,
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key    
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

hotel_rows = []

for index, row in df2.iterrows():  

    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000  # 10km

    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5

    # Set up a parameters dictionary
    params = {
        "categories": categories,
        "limit": limit,
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key    
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # Normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
    
            # Return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website,
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
            "city": city,
            "country": country,
            "latitude": latitude,
            "longitude": longitude,
            "address": None,
            "name": None,
            "distance": None,
            "elevation": None,
            "website": None
        }

    # Append to hotel list
    hotel_rows.append(hotel_row)

Starting hotel search


In [66]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,Puerto Ayora,EC,-0.7393,-90.3518,"Hostal Mirada Del Solitario George Hostel, Pet...",Hostal La Mirada De Solitario George,3776.0,None,None
1,Wailua Homesteads,US,22.0669,-159.3780,"Hilton Garden Inn Kauai Wailua Bay, Kuhio High...",Hilton Garden Inn Kauai Wailua Bay,4923.0,None,None
2,Muisne,EC,0.6000,-80.0333,"Hostal Té de Menta, Manabi, Muisne, Ecuador",Hostal Té de Menta,1146.0,None,None
3,Meulaboh,ID,4.1363,96.1285,"Hotel Meuligou, Jalan Singgah Mata II, Meulabo...",Hotel Meuligou,889.0,None,None
4,Bontang,ID,0.1333,117.5000,"Hotel Sederhana, Jalan Jend. Achmad Yani, Bont...",Hotel Sederhana,1128.0,None,None
...,...,...,...,...,...,...,...,...,...
82,Sisa,PE,-6.6167,-76.6915,None,None,NaN,None,None
83,Nkoteng,CM,4.5167,12.0333,None,None,NaN,None,None
84,Palopo,ID,-2.9996,120.1921,"Hotel Mulia Indah, Jalan Mangga, Palopo 91923,...",Hotel Mulia Indah,511.0,None,None
85,Prey Veng,KH,11.4868,105.3253,"Thuna hotel, National Road 11, Prey Veng, Camb...",Thuna hotel,112.0,None,None


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [67]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name"]  # Add 'city' to the tooltip
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name)